In [1]:
import numpy as np
import scipy

# Question - 1

### Part - a
The states are as follows
$$\{S,1,7,3,8,5,6,W\}$$
This is because we can club together equivalent states, which are 
$$\{2 \equiv 7\}, \{3 \equiv 9\}, \{4 \equiv 8\}$$

The State transition matrix is stated below

$$
P = \begin{bmatrix}
                  0 & 0.25 & 0.25 & 0.25 & 0.25 & 0 & 0 & 0 \\
                  0 & 0 & 0.25 & 0.25 & 0.25 & 0.25 & 0 & 0 \\
                  0 & 0 & 0.25 & 0.25 & 0.25 & 0 & 0 & 0.25 \\
                  0 & 0 & 0.25 & 0 & 0.25 & 0.25 & 0.25 & 0 \\
                  0 & 0 & 0 & 0.25 & 0.5 & 0 & 0 & 0.25 \\
                  0 & 0 & 0.25 & 0.25 & 0.25 & 0 & 0.25 & 0 \\
                  0 & 0 & 0.25 & 0.25 & 0.25 & 0 & 0 & 0.25 \\
                  0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
    \end{bmatrix}
$$
Note that the states in transition matrix are in the order of states written above.

### Part-b
The reward function would be 
$$
R(s) = \begin{cases}
                        0 & \text{if } s = W \\
                        -1 & \text{otherwise}
       \end{cases}
$$
In vector form it could be represented as
$$
R = \begin{pmatrix}
                -1 \\
                -1 \\
                -1 \\
                -1 \\
                -1 \\
                -1 \\
                -1 \\
                0
    \end{pmatrix}
$$
The discount factor should be 1 because our goal is to find the expected number of moves.

*Why?* The reward should be -1 for all the states except for the winning state because our goal is to find the average number of steps to reach the goal state, while our discount factor should be 1 again for the same reason.

Now we can use the matrix form derived from the Bellman equation to solve for the value vector

$$ V = \left( I - \gamma P \right)^{-1}R $$

The Calculation can be found in the cell below.

The expected number of die throws should be slightly above 7.08 from the starting state.

## Why can we make the probability of transition from absorbing state back to itself 0?
The value function evaluated at the absorbing state is just the reward at that state, if there are $N$ states and $1$ absorbing states. We have already evaluated the value function at the absorbing state, now to find the value at other states we set the probability of transition from absorbing state back to itself to 0. This helps reducing the equations to $\left(N-1\right)$ in $V = R + \gamma P V$ and also accounts for the value at absorbing state (which is same as the reward at that state).
**How does it solve the problem of singularity?**   
$$V = R + \gamma P V$$
The above equation is derived from 
$$ V(s) = \text{E}\left[r_{t+1}|s_t=s\right]+\gamma\sum_{s'\in S}P_{ss'}V(s')$$

Notice that when $s$ is the absorbing state, the summation on the right becomes 0, because we won't evaluate it any further instead of defining the Value recursively as a function of value at absorbing state itself. This problem can be avoided by setting $P_{ss}$ to 0 while solving the Bellman equation where $s$ is the absorbing state.

We can consider an example.     
Consider a two state MRP with states $\{A, B\}$, the transition matrix being
$$
\begin{bmatrix}0.1 & 0.9 \\ 0 & 1\end{bmatrix}
$$ 

If we write the equation for value functions we get
\begin{align}
V(A) &= R(A) + 0.1\times V(A) + 0.9 \times V(B)\\
V(B) &= R(B)
\end{align}
This is why we need to make the transition probability from B to B 0, while using the matrix approach. If we don't the matrix approach would be equivalent to
\begin{align}
V(A) &= R(A) + 0.1\times V(A) + 0.9 \times V(B)\\
V(B) &= R(B) + V(B)
\end{align}
which is wrong and is the cause of invertibility, this would happend in any finite horizon MRP.



In [2]:
gamma = 1

# Reward Vector
R = -np.ones(8)
R[-1] = 0

# transition Matrix
P = np.array(
            [[0 , 0.25 , 0.25 , 0.25 , 0.25 , 0 , 0 , 0],
             [0 , 0 , 0.25 , 0.25 , 0.25 , 0.25 , 0 , 0],
             [0 , 0 , 0.25 , 0.25 , 0.25 , 0 , 0 , 0.25],
             [0 , 0 , 0.25 , 0 , 0.25 , 0.25 , 0.25 , 0],
             [0 , 0 , 0 , 0.25 , 0.5 , 0 , 0 , 0.25],
             [0 , 0 , 0.25 , 0.25 , 0.25 , 0 , 0.25 , 0],
             [0 , 0 , 0.25 , 0.25 , 0.25 , 0 , 0 , 0.25],
             [0 , 0 , 0 , 0 , 0 , 0 , 0 , 0]]
            )

def ValueVector(P: np.ndarray, R: np.ndarray, gamma: float) -> np.ndarray:
    """
    Given transition matrix, reward vector and discount factor this
    function calculates the value vector

    Args:
    -----
        P (np.ndarray): transition matrix
        R (np.ndarray): Reward vector
        gamma (int): the discount factor

    Returns:
    --------
        np.ndarray: the value vector
    """
    # The identity matrix
    I = np.eye(*P.shape)
    return scipy.linalg.inv((I - gamma*P)).dot(R)

V = ValueVector(P, R, 1.0)
print(f'The expected number of moves from each state is: {-V}')

The expected number of moves from each state is: [ 7.08333333  7.          5.33333333  6.66666667  5.33333333  6.66666667
  5.33333333 -0.        ]


-----------------
# Question-2

State Space: It can be denoted by the number of machines that are working.

$$ S := \{0, 1 ,\dots, N\} $$

Action Space: 
- It is either don't do anything,
- or get the machines repaired

Lets say action $a_1$ is to not to anything, while action $a_2$ is to get the machines repaired.

Rewards: It has two components -
- k if total of k machines are working
- $-N/2$ if repair is done else 0

$$R(S_{t+1} = S|S_t=k, a_t = a_1) = k$$
$$R(S_{t+1} = S|S_t=k, a_t = a_2) = k- N/2$$

The transition Matrix 
$$P^{a_1} = $$
| $m$ | $n=0$ | $n=1$| $\dots$ | $n=N$ |
| --- | ---- | ---- | ---- | ---- |
|m=0| $1$ | 0 | $\dots$ | 0 |
|m=1| $\frac{1}{2}$ | $\frac{1}{2}$ | $\dots$ | 0 |
| $\vdots$ | $\vdots$ | $\vdots$ | $\ddots$ | $\vdots$ |
| $m = N$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$| 


The transition Matrix 
$$P^{a_2} = $$
| $m$ | $n=0$ | $n=1$| $\dots$ | $n=N-1$ |$n=N$ |
| --- | ---- | ---- | ---- | ---- | ---- |
|m=0| $0$ | 0 | $\dots$ | 0 | 1 | 
|m=1| 0 | 0 | $\dots$ | $\frac{1}{2}$ | $\frac{1}{2}$ |
| $\vdots$ | $\vdots$ | $\vdots$ | $\ddots$ | $\vdots$ | $\vdots$ |
| $m = N$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$ | $\frac{1}{N+1}$| $\frac{1}{N+1}$|

The reward matrix is given by
$$ R^{a_1} = \begin{bmatrix}
                0 & 0 & \dots & 0 \\
                1 & 1 & \dots & 1 \\
                \vdots & \vdots & \ddots & \vdots \\
                N-1 & N-1 & \dots & N-1 \\
                N & N & \dots & N 
             \end{bmatrix}
$$

$$ R^{a_2} = \begin{bmatrix}
                -\frac{N}{2} & -\frac{N}{2} & \dots & -\frac{N}{2}\\
                1-\frac{N}{2} & 1-\frac{N}{2} & \dots & 1-\frac{N}{2} \\
                \vdots & \vdots & \ddots & \vdots \\
                N-1-\frac{N}{2} & N-1-\frac{N}{2} & \dots & N-1-\frac{N}{2} \\
                N-\frac{N}{2} & N-\frac{N}{2} & \dots & N-\frac{N}{2} 
             \end{bmatrix}
$$

## Part-b
The given setting doesn't have an absorbing state, it is an infinite horizon MDP, which means we should use a discounted setting.

## Part-c
Now, we would follow a policy where only action $a_1$ is taken. 

The state space is given by: $\{0, 1, 2, 3, 4, 5\}$ which represents the number of machines working.

The transition matrix is given by

$$ P^{\pi} = \begin{bmatrix}
               1 & 0 & 0 & 0 & 0 & 0 \\
               \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0 \\
               \frac{1}{3} & \frac{1}{3} & \frac{1}{3} & 0 & 0 & 0 \\
               \frac{1}{4} & \frac{1}{4} & \frac{1}{4} & \frac{1}{4} & 0 & 0 \\
               \frac{1}{5} & \frac{1}{5} & \frac{1}{5} & \frac{1}{5} & \frac{1}{5} & 0 \\
               \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} \\
       \end{bmatrix}
$$

$$ R^{\pi}(s) = \sum \pi(a|s) \sum_{s'} P^{a}_{SS'}R^{a}_{SS'}$$

Where $\pi(a|s)$ is always 1, as the probability of taking action $a_1$ is 1.

$$ R^{\pi}(s) = \sum_{s'} P^{a_1}_{SS'}R^{a_1}_{SS'}$$

The reward matrix is given as

$$
R = \begin{bmatrix}
         0 & 0 & 0 & 0 & 0 & 0 \\
         1 & 1 & 1 & 1 & 1 & 1 \\
         2 & 2 & 2 & 2 & 2 & 2 \\
         3 & 3 & 3 & 3 & 3 & 3 \\
         4 & 4 & 4 & 4 & 4 & 4 \\
         5 & 5 & 5 & 5 & 5 & 5
    \end{bmatrix}
$$

We can either calculate the Reward vector using the equation written above, or we can directly say that the reward for being in state S is same as the number of machines that are correctly working on that day.

$$
R^{\pi} = \begin{pmatrix}
            0 \\ 1 \\ 2 \\ 3 \\ 4 \\ 5
          \end{pmatrix}
$$

In [7]:
def generate_subarray(how_many_div: int, size: int) -> np.ndarray:
    res = np.zeros(size)
    for i in range(how_many_div):
        res[i] = 1/how_many_div
    return res
# Now we have a finite horizon MDP, so we can use discount factor as 1

P = np.array([generate_subarray(i+1, 6) for i in range(6)])
P[0, 0] = 0

R = np.array([0, 1, 2, 3, 4, 5])
gamma = 1

V = ValueVector(P, R, gamma)
print(f"The value vector is {V}")

The value vector is [ 0.  2.  4.  6.  8. 10.]


$$\pi'(s) = greedy(V)$$

$$
\pi'(s) = \begin{cases}
                1 & \text{if } a = \text{argmax}_{a \in A} \displaystyle  \left[\sum_{s' \in S} P_{ss'}^a (R_{ss'}^a+\gamma V^\pi(s'))\right] \\ 
                0 & \text{otherwise}
            \end{cases}
$$

Now, lets perform the iteration for each state, we would compare the Value inside the summation for each action, lets call the value as Q
1. for $s = 0$
    - Action $a_1$: $Q(0, a_1)$ = $0$
    - Action $a_2$: $Q(0, a_2)$ = $7.5$   
    Take Action $a_2$
2. For $s = 1$
    - Action $a_1$: $Q(1, a_1)$ = $2$
    - Action $a_2$: $Q(1, a_2)$ = $7.5$   
    Take Action $a_2$
3. For $s = 2$
    - Action $a_1$: $Q(2, a_1)$ = $4$
    - Action $a_2$: $Q(2, a_2)$ = $7.5$   
    Take Action $a_2$
4. For $s = 3$
    - Action $a_1$: $Q(3, a_1)$ = $6$
    - Action $a_2$: $Q(3, a_2)$ = $7.5$   
    Take Action $a_2$
4. For $s = 4$
    - Action $a_1$: $Q(4, a_1)$ = $8$
    - Action $a_2$: $Q(4, a_2)$ = $7.5$   
    Take Action $a_1$
5. For $s = 5$
    - Action $a_1$: $Q(5, a_1)$ = $10$
    - Action $a_2$: $Q(5, a_2)$ = $7.5$   
    Take Action $a_1$
        

--------------------------

# Question-3
State Space S := $\{A, B, C, D\}$

### Policy $\pi_1$
$$P^{\pi_1}(S'|S) = \sum_{a \in A} \pi(a|s) P_{SS'}^a$$

$$
P^{\pi_1} = \begin{bmatrix}
                        0 & 0.9 & 0.1 & 0 \\
                        0.1 & 0 & 0 & 0.9 \\
                        0.9 & 0 & 0 & 0.1 \\
                        0 & 0 & 0 & 1 \\
            \end{bmatrix}                          
$$

$$R^{\pi_1}(s) = \sum \pi_1(a|s) \sum_{s'}P_{ss'}^aR_{ss'}^a$$

We aren't given the rewards in such a format, but for every state apart from the goal state (D) the reward is $-10$, while for the goal state it is $100$

$$R^{\pi_1} = \begin{pmatrix} -10 \\ -10 \\ -10 \\ 100 \end{pmatrix}$$

### Policy $\pi_2$
$$P^{\pi_2}(S'|S) = \sum_{a \in A} \pi(a|s) P_{SS'}^a$$

$$
P^{\pi_2} = \begin{bmatrix}
                        0 & 0.1 & 0.9 & 0 \\
                        0.9 & 0 & 0 & 0.1 \\
                        0.1 & 0 & 0 & 0.9 \\
                        0 & 0 & 0 & 1 \\
            \end{bmatrix}                            
$$

The reward vector would remain that same that is
$$
R^{\pi_2}(s) = \sum \pi_2(a|s) \sum_{s'}P_{ss'}^aR_{ss'}^a
$$
We get
$$
R^{\pi_2} = \begin{pmatrix} -10 \\ -10 \\ -10 \\ 100 \end{pmatrix}
$$

### Policy $\pi_3$
- Action $a_1$ is chosen in states B and D with probability $1.0$
- Action $a_2$ is chosen in state C with probability $1.0$
- Action $a_1$ is chosen in state A with probability $0.4$ and action $a_2$ is chosen with probability $0.6$

$$P^{\pi_3}(S'|S) = \sum_{a \in A} \pi(a|s) P_{SS'}^a$$
$$
P^{\pi_3} = \begin{bmatrix}
                        0 & 0.42 & 0.58 & 0 \\
                        0.1 & 0 & 0 & 0.9 \\
                        0.1 & 0 & 0 & 0.9 \\
                        0 & 0 & 0 & 1 \\
            \end{bmatrix}                          
$$

The reward vector would remain that same that is
$$
R^{\pi_3}(s) = \sum \pi_2(a|s) \sum_{s'}P_{ss'}^aR_{ss'}^a
$$
We get
$$
R^{\pi_3} = \begin{pmatrix} -10 \\ -10 \\ -10 \\ 100 \end{pmatrix}
$$

In [4]:
# Policy 1
P1 = np.array([
               [0, 0.9, 0.1, 0],
               [0.1, 0, 0, 0.9],
               [0.9, 0, 0, 0.1],
               [0 ,0 ,0, 0]
              ])
R1 = np.array([-10, -10, -10, 100])
gamma = 1

V1 = ValueVector(P1, R1, 1)

# Policy 2
P2 = np.array([
               [0, 0.1, 0.9, 0],
               [0.9, 0, 0, 0.1],
               [0.1, 0, 0, 0.9],
               [0 ,0 ,0, 0]
              ])
R2 = R1

V2 = ValueVector(P2, R2, 1)

# Policy 3
P3 = np.array([
               [0, 0.42, 0.58, 0],
               [0.1, 0, 0, 0.9],
               [0.1, 0, 0, 0.9],
               [0 ,0 ,0, 0]
              ])
R3 = R1

V3 = ValueVector(P3, R3, 1)

In [5]:
print("a.) The value vectors for all the policies are")
print(f'V1: {V1}\nV2: {V2}\nV3: {V3}')

a.) The value vectors for all the policies are
V1: [ 75.6097561   87.56097561  68.04878049 100.        ]
V2: [ 75.6097561   68.04878049  87.56097561 100.        ]
V3: [ 77.77777778  87.77777778  87.77777778 100.        ]


**b.)** The policy $\pi_3$ is the best policy among the given policies, because the $\pi_3 \geq \pi_1$ and $\pi_3 \geq \pi_2$ (partial ordering).

**c.)** The policies $\pi_1$ and $\pi_2$ are not comparable since we are not given a starting state and we cannot use poset since the value at B is higher in policy $\pi_1$ while at C is higher in policy $\pi_2$. The policy $\pi_3$ is better than both of the other policies.

**d.)** We can use a greedy approach. Basically for each state look which policy gives a higher value function at that state. Choose that policy. 

$$\pi(s) = \begin{cases}
\pi_1(s) & \text{if } V^{\pi_1}(s) > V^{\pi_2}(s) \\
\pi_2(s) & \text{otherwise}
\end{cases}
$$
This ensures that at each state the value function evaluated at that state would be at least as high as the max of the value functions evaluated using the given two policies.

Essentially, we are choosing the best of both worlds.

----------------------
# Question-4
### Case #1: $\gamma = 0.1$, $\eta = 0$ (low discount factor, low noise)
When the discount factor is small, we are giving preference to the immediate rewards. We are also assuming that there is no noise in the environment. The agent would prefer the close exit that is the state with **$+1$** reward because the discount factor is rather low, achieving the state with **$+10$** would take two more moves and when discounted with $0.1$ it would give an overall lower incentive. Therefore, for low discount factor $\gamma$ and noise $\eta$, the agent would tend to choose the close exit and would even risk the cliff, which is the dotted path in the given image.

### Case #2: $\gamma = 0.1$, $\eta = 0.5$ (low discount factor, high noise)
Now, the discount factor is still small, but the environment is highly stochastic. The agent would still be short-sighted but now it would not risk falling down the cliff and would therefore take the long path, that is the solid line to the state with reward **$+1$**, it wouldn't take the state with reward **$+10$** again because the overall incentive is lower. Therefore, for low discount factor $\gamma$ but high noise $\eta$ the agent would tend to choose the close exit but not risk falling down the cliff, which is the path denoted by solid line.

### Case #3: $\gamma = 0.9$, $\eta = 0$ (high discount factor, low noise)
The discount factor is large, but there is no noise in the environment. The agent would choose a higher reward even if it comes later down the line. In other words, the agent is far-sighted. The agent would tend to choose the distant exit because of the higher reward associated with it and would risk the cliff because the environment is deterministic. In other words, it would choose the dotted line to the state with reward **$+10$**

### Case #4: $\gamma = 0.9$, $\eta = 0.5$ (high discount, high noise)
The discount factor is large, and the environment is also stochastic. The agent would still choose a higher reward even if it comes later down the line. The agent is far-sighted. The agent would tend to choose the distant exit because of the higher reward associated with it but it wouldn't risk falling down the cliff. In other words, it would choose the solid line to the state with reward **$+10$**


----------------------
# Question-5
## Part-a
Yes, it is possible to get $Q^{\pi}_3(s)$ using $Q^{\pi}_2(s)$ and $Q^{\pi}_1(s)$   
First we will prove additivity in Reward vectors
\begin{align}
R^\pi(s) &= \sum_{a \in A} \pi(a|s) \sum_{s' in S} P_{ss'}^aR_{ss'}^a \\
R_3^\pi(s) &= \sum_{a \in A} \pi(a|s) \sum_{s' in S} P_{ss'}^a(R_{1_{ss'}}^a +R_{2_{ss'}}^a) \\
R_3^\pi(s) &= R_1^\pi(s) + R_2^\pi(s)
\end{align}
Now, we will prove that the value functions follow additivity.
From the Bellman equation, we know that 
$$V^\pi = \left(I - \gamma P\right)^{-1}R^\pi$$
We can write
$$V_3^\pi = \left(I - \gamma P^\pi\right)^{-1}R_3^\pi = \left(I - \gamma P^\pi\right)^{-1}\left(R_1^\pi+R_2^\pi\right) = V_1^\pi + V_2^\pi$$

Now, we can use
$$Q^\pi(s,a) = \sum_{s' \in S} P_{ss'}^a\left[\gamma V^\pi(s') + R_{ss'}^a\right]$$

Now, we can write it as
\begin{align}
Q_3^\pi(s,a) &= \sum_{s' \in S} P_{ss'}^a\left[\gamma V_3^\pi(s') + R_{3_{ss'}}^a\right] \\
&= \sum_{s' \in S} P_{ss'}^a\left[\gamma \left(V_1^\pi(s')+V_2^\pi(s')\right) + R_{1_{ss'}}^a+R_{2_{ss'}}^a\right]\\
&= Q_1^\pi(s,a)+Q_2^\pi(s,a)
\end{align}

## Part-b
We cannot combine the two optimal policies $\pi_1^*$, $\pi_2^*$, this is because the action that the optimal policy would take may not come from either of the two policies. Max operator is non-linear and we won't get a simple relation.

## Part-c 
We can use the result from part-a, that
$$V^{\pi}_3 = V^{\pi}_1 + V^{\pi}_2$$

Now, consider that a policy better than $\pi^*$ (at least on 1 state) exists for $M_3$, lets call this policy $\pi^{'*}$, we know

$$V^{\pi^{'*}}_3 = V^{\pi^{'*}}_2 + V^{\pi^{'*}}_1$$

We also know that 
$$V^{\pi^{'*}}_3 \geq V^{\pi^{*}}_3$$ 
since $\pi^{'*}$ is the optimal policy for $M_3$

Therefore at least one of the two must hold
1. policy $\pi^{'*}$ is better than $\pi^{*}$ for $M_1$
2. policy $\pi^{'*}$ is better than $\pi^{*}$ for $M_2$

But we know that the policy $\pi^{*}$ is optimal for both $M_1$ and $M_2$ which is a contradiction. 
$\therefore$ the policy $\pi^{*}$ is optimal for $M_3$

## Part-d
FIrst we will find a relation between the reward vectors

\begin{align}
R_1^{\pi}(s) &= \sum_{a \in A} \pi(a|s) \sum_{s' \in S}P_{ss'}^aR_{1_{ss'}}^a \\
&= \sum_{a \in A} \pi(a|s) \sum_{s' \in S}P_{ss'}^a(R_{1_{ss'}}^a+\epsilon)\\
&= \sum_{a \in A} \pi(a|s) \sum_{s' \in S}P_{ss'}^a\epsilon + R_2^{\pi}(s)\\
&= \epsilon \sum_{a \in A} \pi(a|s) + R_2^{\pi}(s)\\
&= \epsilon + R_2^{\pi}(s)
\end{align}

Now, we can use bellman equation
\begin{align}
V_1^{\pi} &= (I - \gamma P)^{-1}R_1^\pi\\
&= (I - \gamma P)^{-1}(R_2^\pi+\epsilon \vec{1})\\
&= V_2^\pi + \epsilon (I-\gamma P)^{-1}\vec{1}
\end{align}

We can simplify this for infinite horizon MDP, assume
$$V_1^\pi = V_2^\pi + \vec{k} $$

From the results obtained above
\begin{align}
k\vec{1} = \epsilon (I-\gamma P)^{-1}\vec{1}\\
k(I-\gamma P)\vec{1} = \epsilon \vec{1}\\
k(\vec{1}-\gamma)\vec{1} = \epsilon \vec{1}\\
k = \frac{\epsilon}{1-\gamma}
\end{align}
Note that this result holds only for infinite horizon MDPs,

### Why?
Because for finite horizon MDPs, we would have to omit the transition probability from absorbing state to itself. An example is given below for finite horizon MDPs.



In [37]:
P = np.array([[0.9, 0.1],[0, 1]])
R = np.array([1,10])
P[-1, -1] = 0
V1 = ValueVector(P, R, 0.25)
V2 = ValueVector(P, R+0.1, 0.5)
print(V1)
print(V2)

The finite horizon MDP case doesn't simplify any further.